# HOUSING MARKET ANALYSIS

In [1]:
from pymongo import MongoClient
import pymongo


In [2]:
# establish connection to the mongodb
client = MongoClient("mongodb://127.0.0.1:27017/")   


In [5]:
# retrieve the database and collections
housing = client["house_new_data"]
market = housing["market"]


In [6]:
# check if connection has beeen successfully established
client.list_database_names()


['admin',
 'config',
 'house',
 'house_data',
 'house_new_data',
 'housing',
 'local']

In [44]:
val = input("Choose the operation you want to perform\n 1.Search\n 2.View \n 3.Compare \n 4.Insert \n 5.Update \n 6.Delete\n\n")

if val=='1':
    val1= input("\nChoose one of the following search operations\n 1.Search for a specific state\n 2.Search for a specific city\n 3.Search in a specific time frame\n 4.Search in a specific price range\n 5.Search by property type\n\n")
    
    if val1=='1':
        state=input("\nEnter state name\n\n")
        result=market.find({'state': state},{'city': 1, 'state': 1, 'region':1,'property_type':1,'median_sale_price':1}).limit(10)
    
    elif val1=='2':
        city=input("\nEnter city name\n\n")
        result=market.find({'city': city},{'city': 1, 'state': 1, 'region':1,'property_type':1,'median_sale_price':1}).limit(10)
        
    elif val1=='3':
        year = input("\nEnter year in the format xxxx\n\n")
        month = input("\nEnter month in the format xx\n\n")
        time = "{}-{}-01".format(year, month)
        result=market.aggregate( [{"$match" : {"period_begin": {"$in" :[time]}}}, {"$limit" : 10 }])

    elif val1=='4':
        low=int(input("\nEnter lower price range\n\n"))
        high=int(input("\nEnter higher price range\n\n"))
        result=market.find({"median_sale_price":{'$gte':low,'$lte':high}},{'region':1,'median_sale_price':1}).limit(10)

    elif val1=='5':
        prop=input("\nEnter property type\n\n")
        result=market.find({"property_type":prop},{'city':1,'state':1,'region':1,'property_type':1,'median_sale_price':1}).limit(10)


elif val=='2':
    
    val2= input("\nChoose one of the following view operations\n 1.Show regions that are within a certain budget\n 2.Show the average median sale price for a specific region\n 3.Show the total number of houses sold in a specific region\n\n")
    
    if val2=='1':
        low2=int(input("\nEnter lower price range\n\n"))
        high2=int(input("\nEnter higher price range\n\n"))
        result=market.aggregate([{"$match":{"median_sale_price":{"$gte":low2,"$lte":high2}}},{"$group":{"_id":"$region","average":{"$avg":"$median_sale_price"}}}, {"$limit" : 10 }])

        
    elif val2=='2':
        city2 = input('\nEnter the city\n\n ')
        state_code = input('\nEnter the state code\n\n ')
        region = "{}, {}".format(city2,state_code)
        result=market.aggregate([{"$match" : {"region" : region }},{ '$group':{ '_id' : '$region', 'Average median sale price' : { '$avg': '$median_sale_price' }}}, {"$limit" : 10 }])
        
    elif val2=='3':
        city2 = input('\nEnter the city \n\n')
        state_code = input('\nEnter the state code \n\n')
        region = "{}, {}".format(city2,state_code)
        result=market.aggregate([{"$match":{"region":region}}, {"$group":{"_id":"$region","total houses sold":{"$sum":"$homes_sold"}}}, {"$limit" : 10 }])


elif val=='3':
    
    val3=input("\nChoose one of the following compare operations\n 1.Compare the average median sale prices between two regions\n 2.Compare the total houses sold between two regions\n\n")
    city1 = input('\nEnter the city of first region \n\n ')
    state_code1 = input('\nEnter state code of first region\n\n ')
    city2 = input('\nEnter the city of second region \n\n')
    state_code2 = input('\nEnter state code of second region \n\n')
    region1 = "{}, {}".format(city1,state_code1)
    region2 = "{}, {}".format(city2,state_code2)
    
    if val3=='1':
        
        result=market.aggregate([{"$match" : {"region": {"$in" : [region1, region2 ]}}}, { '$group':{ '_id' : '$region', 'sum' : { '$avg': '$median_sale_price' }}} ])
        
    elif val3=='2':
        result=market.aggregate( [{"$match" : {"region": {"$in" : [region1,region2 ]}}}, { '$group':{ '_id' : '$region', 'Total houses sold' : { '$sum': '$homes_sold' }}} ])
        
elif val=='4':
    
    inp_str= input("\nEnter the following required fields in the same order separated by comma -> \n\nBeginning period \nEnd period \nDuration of period \nRegion type \nRegion type id \nTable id \nIs it seasonally adjusted? ( T/F ) \nRegion (city, state code) \nCity \nState \nState code \nProperty type \nProperty type id \nMedian sales price \nMedian list price \nMedian ppsf \nMedian list ppsf \nHome sold \nPending sales \nNew listing \nInventory \nMonths of supply \nMedian days on market \nMedian dom (mom) \nMedian dom (yoy) \nAverage sale to list \nSold above list \nPrice drops \nOff market in two weeks \nParent metro region \nParent metro region code \nLast updated (date, time) \n\n ")
    values=inp_str.split(',')
    period_begin, period_end, period_duration, region_type, region_type_id, table_id, is_seasonally_adjusted, region, city, state,  state_code, property_type,  property_type_id, median_sale_price, median_list_price, median_ppsf, median_list_ppsf, homes_sold, pending_sales, new_listings, inventory, months_of_supply, median_dom, median_dom_mom, median_dom_yoy, avg_sale_to_list, sold_above_list, price_drops, off_market_in_two_weeks, parent_metro_region, parent_metro_region_metro_code, last_update = values
    
    period_duration =int(period_duration)
    region_type_id=int(region_type_id)
    table_id=int(table_id)
    median_sale_price=int(median_sale_price)
    median_list_price=int(median_list_price)
    median_ppsf=float(median_ppsf)
    median_list_ppsf=float(median_list_ppsf)
    homes_sold=int(homes_sold)
    pending_sales=int(pending_sales)
    new_listings=int(new_listings)
    median_dom=int(median_dom)
    median_dom_mom=float(median_dom_mom)
    median_dom_yoy=float(median_dom_yoy)
    avg_sale_to_list=float(avg_sale_to_list)
    sold_above_list=int(sold_above_list)
    off_market_in_two_weeks=int(off_market_in_two_weeks)
    parent_metro_region_metro_code=int(parent_metro_region_metro_code)
    
    new_data = { "period_begin": period_begin, "period_end": period_end,  "period_duration": {"$numberInt": period_duration}, 
                'region_type': region_type, 'region_type_id': {'$numberInt': region_type_id},  
                'table_id': {'$numberInt':table_id}, 'is_seasonally_adjusted': is_seasonally_adjusted,  
                'region': region, 'city': city, 'state': state, 'state_code': state_code,  
                'property_type': property_type, 'property_type_id':property_type_id,
                'median_sale_price':{'$numberInt':median_sale_price}, 'median_list_price':{'$numberInt':median_list_price},  
                'median_ppsf':{'$numberDecimal': median_ppsf},  'median_list_ppsf': {'$numberDecimal':median_list_ppsf}, 
                'homes_sold': {'$numberInt': homes_sold}, 'pending_sales': {'$numberInt': pending_sales},  
                'new_listings': {'$numberInt': new_listings}, 'inventory':inventory, 'months_of_supply': months_of_supply, 
                'median_dom': {'$numberInt': median_dom}, 'median_dom_mom': {'$numberDecimal': median_dom_mom}, 'median_dom_yoy': {'$numberDecimal': median_dom_yoy},  'avg_sale_to_list':{'$numberDecimal': avg_sale_to_list}, 
                'sold_above_list': {'$numberInt': sold_above_list}, 'price_drops': price_drops, 
                'off_market_in_two_weeks': {'$numberInt':off_market_in_two_weeks }, 'parent_metro_region': parent_metro_region,  
                'parent_metro_region_metro_code': {'$numberInt': parent_metro_region_metro_code}, 'last_updated': last_update }
                
    res=market.insert_one(new_data)
    print("\nData inserted successfully.\nInsert ID ->",res.inserted_id,"\n\n")
    print(market.find_one({'_id': res.inserted_id}))

elif val=='5':
    
    val5= input("\nChoose the update operation\n1.Increase listing price for specific region by some percent\n\n")
        
    if val5=='1':
        city = input('\nEnter the city\n\n ')
        state_code = input('\nEnter the state code\n\n ')
        region = "{}, {}".format(city,state_code)
        per=input("\nEnter percentage increase\n")
        percent=int(per)/100
        result=market.aggregate([{'$match': {'region': region}},{'$project': {'region':1,'property_type':1,'median_sale_price':1, 'median_list_price': {'$add': ['$median_list_price', {'$multiply': ["$median_list_price", percent]}]}}}])
        
        
elif val=='6':
    
    val6= input("\n Choose one of the following delete operations\n 1.Remove all data of a specific region\n 2.Remove all data belonging to a specific time\n 3.Remove a field in the dataset\n\n")
     
    if val6=='1':
        city = input('\nEnter the city\n\n ')
        state_codeq = input('\nEnter the state code\n\n')
        region = "{}, {}".format(city,state_code)        
        market.delete_many({'region':region})  
        result=market.find({'region':region}).limit(10)
        print("\nDeleted all data belonging to ",region)
        
    elif val6=='2':
        year = input("\nEnter year in the format xxxx\n\n")
        month = input("\nEnter month in the format xx\n\n")
        time = "{}-{}-01".format(year, month)
        market.delete_many({'period_begin':time})
        result=market.find({'period_begin':time}).limit(10)
        print("\nDeleted all data from ",time)
        
    elif val6=='3':
        field=input("\nEnter the field to be deleted\n\n")
        market.update_many({},{"$unset":{field:1}})
        result=market.find().limit(10)
        print("\nSucessfully deleted ",field," from the dataset\n")
        
print("\n")
for i in result:
    print(i,"\n")
    
        

Choose the operation you want to perform
 1.Search
 2.View 
 3.Compare 
 4.Insert 
 5.Update 
 6.Delete

2

Choose one of the following view operations
 1.Show regions that are within a certain budget
 2.Show the average median sale price for a specific region
 3.Show the total number of houses sold in a specific region

3

Enter the city 

San Diego

Enter the state code 

CA


{'_id': 'San Diego, CA', 'total houses sold': 318632.0} 

